In [ ]:
# Cell # 1
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# Cell # 2
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

In [ ]:
# Cell # 3
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

# Convert categorical data to numbers
le_sex = LabelEncoder()
le_smoker = LabelEncoder()
le_region = LabelEncoder()

dataset['sex'] = le_sex.fit_transform(dataset['sex'])
dataset['smoker'] = le_smoker.fit_transform(dataset['smoker'])
dataset['region'] = le_region.fit_transform(dataset['region'])

# Separate features and target
train_dataset = dataset.drop('expenses', axis=1)
train_labels = dataset.pop('expenses')

# Split into train and test datasets (80% train, 20% test)
train_dataset, test_dataset, train_labels, test_labels = train_test_split(
    train_dataset, train_labels, test_size=0.2, random_state=42
)

# Normalize features
scaler = StandardScaler()
train_dataset = scaler.fit_transform(train_dataset)
test_dataset = scaler.transform(test_dataset)

# Create the model
model = tf.keras.Sequential([
    layers.Dense(units=1, input_shape=[train_dataset.shape[1]])
])

# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),
    loss='mean_absolute_error',
    metrics=['mae', 'mse']
)

# Train the model
history = model.fit(
    train_dataset, train_labels,
    epochs=100,
    validation_split=0.2,
    verbose=0
)

# Print training results
print("Training completed. Final training MAE:", history.history['mae'][-1])

In [ ]:
# Cell # 4
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
